In [1]:
# pip
# !pip install tqdm
# !pip install dask

In [2]:
# conda
# !conda install tqdm
# !conda install dask

### Importing necessary libraries

In [3]:
# configuration, worked on using python@3.10.9 
import os
import urllib
import json
from threading import Thread, Lock
from tqdm import tqdm
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, BooleanType
from haversine import haversine, Unit

from leafmap import leafmap
import geopandas as gpd

### Dataset links and filename

In [4]:
# data folder
data_dir = 'data'
intr_dir = 'results'

# data urls
historic_arrest_loc = { 'url': 'https://data.cityofnewyork.us/resource/8h9b-rp9u.json?$limit=15000000', 'filename': 'arrest.json' }
historic_complaint_loc = { 'url': 'https://data.cityofnewyork.us/resource/qgea-i56i.json?$limit=15000000', 'filename': 'complaint.json' }
historic_court_summons_loc = { 'url': 'https://data.cityofnewyork.us/resource/sv2w-rv3k.json?$limit=15000000', 'filename': 'summons.json' }
traffic_speed_loc = { 'url': 'https://data.cityofnewyork.us/resource/i4gi-tjb9.json?$limit=15000000', 'filename': 'speed.json' }
turnstile_loc = { 'url': 'https://data.ny.gov/resource/i55r-43gk.json?$limit=15000000', 'filename': 'turnstile.json' }
subway_loc = { 'url': 'http://web.mta.info/developers/data/nyct/subway/Stations.csv?$limit=10000', 'filename': 'subway.csv' }

### Dataset: Downloading handler

In [5]:
# download flags
downloadflag = True
redownload = False

thread_lock = Lock()

# download utils
def download_dataset_thread(loc, folder):
    with thread_lock:
        if ((not os.path.exists(os.path.join(folder, loc['filename']))) or redownload) and downloadflag:
            if os.path.isfile(os.path.join(folder, loc['filename'])):
                os.remove(os.path.join(folder, loc['filename']))
            if not os.path.exists(folder):
                os.makedirs(folder) 
            with tqdm(unit="B", unit_scale=True, desc=loc['filename'], miniters=1) as progress_bar:
                urllib.request.urlretrieve(loc['url'], os.path.join(folder, loc['filename']), lambda block_num, block_size, total_size: progress_bar.update(block_size))
            progress_bar.display()
        
def download_dataset(loc, folder):
    thread = Thread(target=download_dataset_thread, args=(loc, folder))
    thread.start()
    thread.join()
        

### Dataset: Downloading

In [6]:
# download datasets
for dataset in [historic_arrest_loc,
                historic_complaint_loc,
                historic_court_summons_loc,
                turnstile_loc,
                subway_loc]:
    download_dataset(dataset, data_dir)

### Creating spark session using SparkSession builder

In [7]:
# spark session initialization
spark = SparkSession.builder\
    .master("local[5]")\
    .appName("main")\
    .config("spark.sql.debug.maxToStringFields", 100)\
    .config("spark.driver.memory", '4g')\
    .config("spark.executor.instances", 5)\
    .getOrCreate()

spark

Setting default log level to "

### Initializing spark dataframes

In [8]:
# dataframes
arrest_df = spark.read.json(os.path.join(data_dir, historic_arrest_loc['filename']), multiLine=True).repartition(5)
complaint_df = spark.read.json(os.path.join(data_dir, historic_complaint_loc['filename']), multiLine=True).repartition(5)
summons_df = spark.read.json(os.path.join(data_dir, historic_court_summons_loc['filename']), multiLine=True).repartition(5)
turnstile_df = spark.read.json(os.path.join(data_dir, turnstile_loc['filename']), multiLine=True).repartition(5)
raw_lookup_df = spark.read.option('header', True).option('inferSchema', True).csv(os.path.join('remote_complex_lookup.csv')).repartition(5)
raw_subway_df = spark.read.option('header', True).option('inferSchema', True).csv(os.path.join(data_dir, subway_loc['filename'])).repartition(5)

### UDFs

In [9]:
# haversine distance comparison udf
def withinMile(slat, slong, dlat, dlong):
    srs = (slat, slong)
    dst = (dlat, dlong)
    distance = float(haversine(srs, dst,unit=Unit.MILES))
    return bool(distance < 0.1)
    
withinMileUdf = F.udf(withinMile, BooleanType())

### Dataset Pre-processing

#### 1. Subway and Lookup cleaning and filtering

In [10]:
raw_subway_df.printSchema()

root
 |-- Station ID: integer (nullable = true)
 |-- Complex ID: integer (nullable = true)
 |-- GTFS Stop ID: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Line: string (nullable = true)
 |-- Stop Name: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Daytime Routes: string (nullable = true)
 |-- Structure: string (nullable = true)
 |-- GTFS Latitude: double (nullable = true)
 |-- GTFS Longitude: double (nullable = true)
 |-- North Direction Label: string (nullable = true)
 |-- South Direction Label: string (nullable = true)
 |-- ADA: integer (nullable = true)
 |-- ADA Direction Notes: string (nullable = true)
 |-- ADA NB: integer (nullable = true)
 |-- ADA SB: integer (nullable = true)
 |-- Capital Outage NB: string (nullable = true)
 |-- Capital Outage SB: string (nullable = true)



In [11]:
lookup_df = raw_lookup_df.select(F.col('remote ').alias('unit'), F.col('complex_id')).dropDuplicates(['unit', 'complex_id'])

In [12]:
subway_df = raw_subway_df.join(lookup_df, F.col('Complex ID') == F.col('complex_id'), 'inner').dropDuplicates(['unit'])

In [13]:
subway_df.printSchema()

root
 |-- Station ID: integer (nullable = true)
 |-- Complex ID: integer (nullable = true)
 |-- GTFS Stop ID: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Line: string (nullable = true)
 |-- Stop Name: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Daytime Routes: string (nullable = true)
 |-- Structure: string (nullable = true)
 |-- GTFS Latitude: double (nullable = true)
 |-- GTFS Longitude: double (nullable = true)
 |-- North Direction Label: string (nullable = true)
 |-- South Direction Label: string (nullable = true)
 |-- ADA: integer (nullable = true)
 |-- ADA Direction Notes: string (nullable = true)
 |-- ADA NB: integer (nullable = true)
 |-- ADA SB: integer (nullable = true)
 |-- Capital Outage NB: string (nullable = true)
 |-- Capital Outage SB: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- complex_id: integer (nullable = true)



#### 2. Arrest dataset cleaning

In [14]:
# preprocess arrest data
arrest_df = arrest_df.select("arrest_boro","arrest_date", "arrest_key", "latitude", "longitude", )\
    .filter(F.col('latitude').isNotNull()\
        & F.col('longitude').isNotNull())\
    .withColumn("longitude", F.col("longitude").cast("double"))\
    .withColumn("latitude", F.col("latitude").cast("double"))
arrest_df.show()

+-----------+--------------------+----------+------------------+------------------+
|arrest_boro|         arrest_date|arrest_key|          latitude|         longitude|
+-----------+--------------------+----------+------------------+------------------+
|          B|2010-03-12T00:00:...|  71730512| 40.82095486800005|-73.85801740899994|
|          M|2009-06-03T00:00:...|  62411068|40.863003221000035|-73.92425384299997|
|          B|2007-12-29T00:00:...|  37340060|40.867263716000025|-73.89685776699997|
|          K|2014-08-29T00:00:...| 138453053| 40.68746505200004|     -73.952855937|
|          K|2009-05-21T00:00:...|  61995801| 40.69395702200006|-73.91183165399997|
|          B|2012-05-30T00:00:...|  85006006|40.831974874000025|-73.89669204099994|
|          M|2016-08-27T00:00:...| 156196665| 40.81021769500006|-73.93949694199993|
|          Q|2016-03-11T00:00:...| 151134432| 40.70010913700002|-73.82421171499993|
|          M|2013-08-07T00:00:...|  92014453| 40.70578749500004|-74.00838356

#### 3. Complaint dataset cleaning

In [15]:
# preprocess complaint data
complaint_df = complaint_df.select("cmplnt_num","loc_of_occur_desc", "cmplnt_to_dt", "cmplnt_to_tm", "latitude", "longitude", )\
    .filter(F.col('latitude').isNotNull()\
        & F.col('longitude').isNotNull())\
    .withColumn("longitude", F.col("longitude").cast("double"))\
    .withColumn("latitude", F.col("latitude").cast("double"))
complaint_df.show()

+----------+-----------------+--------------------+------------+------------------+------------------+
|cmplnt_num|loc_of_occur_desc|        cmplnt_to_dt|cmplnt_to_tm|          latitude|         longitude|
+----------+-----------------+--------------------+------------+------------------+------------------+
| 870010794|             null|2020-03-06T00:00:...|    01:00:00| 40.68900458000007|-73.78580605399996|
| 532313042|         FRONT OF|2020-02-17T00:00:...|    20:36:00| 40.71224785000004|-73.76903757999997|
| 106900823|           INSIDE|2006-02-12T00:00:...|    17:50:00|      40.699166247|     -73.792066606|
| 408535776|           INSIDE|2006-04-11T00:00:...|    10:56:00|      40.680332821|     -73.905972878|
| 583339129|             null|                null|        null|      40.688378278|      -73.94491166|
| 562935534|         FRONT OF|2017-05-27T00:00:...|    21:30:00|        40.8551677|      -73.88790442|
| 800736634|             null|2014-01-03T00:00:...|    20:00:00|      40.

#### 4. Summons dataset cleaning

In [16]:
# preprocess summons data
summons_df = summons_df.select("jurisdiction_code","law_description", "offense_description", "precinct_of_occur", "latitude", "longitude", )\
    .filter(F.col('latitude').isNotNull()\
        & F.col('longitude').isNotNull())\
    .withColumn("longitude", F.col("longitude").cast("double"))\
    .withColumn("latitude", F.col("latitude").cast("double"))
summons_df.show()

+-----------------+-------------------+--------------------+-----------------+------------------+------------------+
|jurisdiction_code|    law_description| offense_description|precinct_of_occur|          latitude|         longitude|
+-----------------+-------------------+--------------------+-----------------+------------------+------------------+
|                0|Administrative Code|VEND:RESTRICTIONS...|              109|40.757105780000074|-73.82896307099996|
|                0|Administrative Code|  VENDOR: UNLICENSED|               94| 40.71812038400003|-73.95743012599996|
|                0|Administrative Code|  VENDOR: UNLICENSED|               94| 40.71812038400003|-73.95743012599996|
|                0|Administrative Code|  VENDOR: UNLICENSED|               94| 40.71812038400003|-73.95743012599996|
|                0|Administrative Code|  VENDOR: UNLICENSED|               94| 40.71812038400003|-73.95743012599996|
|                0|Administrative Code|  VENDOR: UNLICENSED|    

### Dataset: Consolidating

1. Combine Subway to Turnstile

In [17]:
# filter and group turnstile dataset
gt_df = turnstile_df.groupBy(F.col('unit'))\
    .agg({ 'entries': 'sum', 'exits': 'sum'})\
        .select(F.col('unit'),\
            F.col('sum(entries)').alias('entries'),\
            F.col('sum(exits)').alias('exits'))
gt_df.show()

+----+-----------------+-----------------+
|unit|          entries|            exits|
+----+-----------------+-----------------+
|R159|  9.7658670459E10|  3.6752142042E10|
|R143| 6.16250900125E11| 7.75773326471E11|
|R469|  1.0314052482E10|     9.79342886E8|
|R167|  1.6874102816E11| 1.96763060056E11|
|R100|  3.8355082149E10|  4.2813609167E10|
|R177| 2.21718982614E11| 1.91598461153E11|
|R290|  7.7285465068E10|  9.9919615555E10|
|R082|  4.0183520438E10|  5.4452586495E10|
|R384|   2.283725805E10|  2.2146211814E10|
|R432|  1.5507768821E10|    7.016570014E9|
|R541|  2.0813861412E10|  1.6886431668E10|
|R046| 4.16017591248E11| 2.68429971561E11|
|R412|    9.997534418E9|     3.88309983E9|
|R018|1.385354956576E12|1.019986375028E12|
|R430|    9.052436119E9|   1.126470821E10|
|R294|  4.0256229752E10|  2.4555010956E10|
|R329|  1.1117120569E10|    5.684716085E9|
|R431|1.094341262102E12|1.096776281978E12|
|R125|  3.7882386986E10|  2.4059386949E10|
|R462| 3.48375809509E11| 3.96053270441E11|
+----+-----

In [18]:
# combine the total exits and entries for each station
st_df = subway_df.join(gt_df, on='unit', how='inner')\
    .select(F.col('GTFS Stop ID').alias('id'),\
        F.col('Line').alias('line'),\
        F.col('Stop Name').alias('stop_name'),\
        F.col('Borough').alias('borough'),\
        F.col('GTFS Latitude').alias('lat'),\
        F.col('GTFS Longitude').alias('long'),\
        F.col('North Direction Label').alias('n_label'),\
        F.col('South Direction Label').alias('s_label'),\
        F.col('entries'),\
        F.col('exits'))

st_df.write.option('header', True).mode('overwrite').csv(os.path.join(intr_dir, 'st_df.csv'))

In [19]:
# get st_df from file
st_df = spark.read.csv(os.path.join(intr_dir, 'st_df.csv'), header=True, inferSchema=True)
st_df.show()

+---+-------------------+--------------------+-------+---------+----------+--------------------+--------------------+-----------------+-----------------+
| id|               line|           stop_name|borough|      lat|      long|             n_label|             s_label|          entries|            exits|
+---+-------------------+--------------------+-------+---------+----------+--------------------+--------------------+-----------------+-----------------+
|117|     Broadway - 7Av|116 St-Columbia U...|      M|40.807722| -73.96411|  Uptown & The Bronx|            Downtown|  9.7658670459E10|  3.6752142042E10|
|633|       Lexington Av|               28 St|      M| 40.74307|-73.984264|  Uptown & The Bronx|            Downtown| 6.16250900125E11| 7.75773326471E11|
|121|     Broadway - 7Av|               86 St|      M|40.788644|-73.976218|  Uptown & The Bronx|            Downtown|  1.6874102816E11| 1.96763060056E11|
|M01|          Myrtle Av|Middle Village-Me...|      Q|40.711396|-73.889601| 

2. Combine Subway to Arrest

In [21]:
# print arrest schema
arrest_df.printSchema()

root
 |-- arrest_boro: string (nullable = true)
 |-- arrest_date: string (nullable = true)
 |-- arrest_key: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [22]:
# perform inner join st_df and arrest_df
sa_df = st_df.join(arrest_df, withinMileUdf(F.col('lat'), F.col('long'), F.col('latitude'), F.col('longitude')), 'inner')\
    .drop(F.col('latitude'))\
    .drop(F.col('longitude'))\
    .na.drop(subset=["stop_name"])\
    .groupBy([F.col('id'),\
              F.col('line'),\
              F.col('stop_name'),\
              F.col('borough'),\
              F.col('lat'),\
              F.col('long'),\
              F.col('n_label'),\
              F.col('s_label'),\
              F.col('entries'),\
              F.col('exits')])\
    .count()\
    .withColumn('arrests', F.col('count'))\
    .drop(F.col('count'))

# write sa_df to file
sa_df.write.option('header', True).mode('overwrite').csv(os.path.join(intr_dir, 'sa_df.csv'))

In [23]:
# fetch saved sa_df and display top rows
sa_df = spark.read.csv(os.path.join(intr_dir, 'sa_df.csv'), header=True, inferSchema=True)
sa_df.show()

+---+-------------------+--------------------+-------+---------+----------+--------------------+--------------------+-----------------+----------------+-------+
| id|               line|           stop_name|borough|      lat|      long|             n_label|             s_label|          entries|           exits|arrests|
+---+-------------------+--------------------+-------+---------+----------+--------------------+--------------------+-----------------+----------------+-------+
|A28| 8th Av - Fulton St|  34 St-Penn Station|      M|40.752287|-73.993391|     Uptown - Queens| Downtown & Brooklyn|1.246297471441E12|4.94987479787E11|  22090|
|725|           Flushing|      Times Sq-42 St|      M|40.755477|-73.987691|              Queens|34 St - Hudson Yards| 4.94357936177E11|1.84005946083E11|  11199|
|D13|          Concourse|              145 St|      M|40.824783|-73.944216|  Uptown & The Bronx| Downtown & Brooklyn| 1.86300028625E11|1.86449494891E11|   6721|
|136|     Broadway - 7Av|         

3. Combine Subway to Criminal

In [24]:
# print complaint schema
complaint_df.printSchema()

root
 |-- cmplnt_num: string (nullable = true)
 |-- loc_of_occur_desc: string (nullable = true)
 |-- cmplnt_to_dt: string (nullable = true)
 |-- cmplnt_to_tm: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [25]:
# join complaints to the sa_df
sc_df = sa_df.join(complaint_df, withinMileUdf(F.col('lat'), F.col('long'), F.col('latitude'), F.col('longitude')), 'inner')\
    .drop(F.col('latitude'))\
    .drop(F.col('longitude'))\
    .na.drop(subset=["stop_name"])\
    .groupBy([F.col('id'),\
              F.col('line'),\
              F.col('stop_name'),\
              F.col('borough'),\
              F.col('lat'),\
              F.col('long'),\
              F.col('n_label'),\
              F.col('s_label'),\
              F.col('entries'),\
              F.col('exits'),\
              F.col('arrests')])\
    .count()\
    .withColumn('complaints', F.col('count'))\
    .drop(F.col('count'))

# write sa_df to file
sc_df.write.option('header', True).mode('overwrite').csv(os.path.join(intr_dir, 'sc_df.csv'))

In [26]:
# fetch saved sa_df and display top rows
sc_df = spark.read.csv(os.path.join(intr_dir, 'sc_df.csv'), header=True, inferSchema=True)
sc_df.show()

+---+--------------------+--------------------+-------+---------+----------+--------------------+--------------------+-----------------+-----------------+-------+----------+
| id|                line|           stop_name|borough|      lat|      long|             n_label|             s_label|          entries|            exits|arrests|complaints|
+---+--------------------+--------------------+-------+---------+----------+--------------------+--------------------+-----------------+-----------------+-------+----------+
|G13|         Queens Blvd|         Elmhurst Av|      Q|40.742454|-73.882017|        Forest Hills|           Manhattan| 6.65829665657E11|1.384726586825E12|   1651|      1954|
|R41|              4th Av|               59 St|     Bk|40.641362|-74.017881|           Manhattan|Coney Island - Ba...|  8.2568001773E10|  6.6395366214E10|   2674|      3121|
|129|      Broadway - 7Av|               28 St|      M|40.747215|-73.993365|  Uptown & The Bronx|            Downtown|  4.83268848

4. Combine Subway to Summons

In [27]:
summons_df.printSchema()

root
 |-- jurisdiction_code: string (nullable = true)
 |-- law_description: string (nullable = true)
 |-- offense_description: string (nullable = true)
 |-- precinct_of_occur: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [28]:
# join summons to the sc_df
ss_df = sc_df.join(summons_df, withinMileUdf(F.col('lat'), F.col('long'), F.col('latitude'), F.col('longitude')), 'inner')\
    .drop(F.col('latitude'))\
    .drop(F.col('longitude'))\
    .na.drop(subset=["stop_name"])\
    .groupBy([F.col('id'),\
              F.col('line'),\
              F.col('stop_name'),\
              F.col('borough'),\
              F.col('lat'),\
              F.col('long'),\
              F.col('n_label'),\
              F.col('s_label'),\
              F.col('entries'),\
              F.col('exits'),\
              F.col('arrests'),\
              F.col('complaints')])\
    .count()\
    .withColumn('summons', F.col('count'))\
    .drop(F.col('count'))

# write sa_df to file
ss_df.write.option('header', True).mode('overwrite').csv(os.path.join(intr_dir, 'ss_df.csv'))

In [29]:
# fetch saved ss_df and display top rows
ss_df = spark.read.csv(os.path.join(intr_dir, 'ss_df.csv'), header=True, inferSchema=True)
ss_df.show()

+---+-------------------+--------------------+-------+---------+----------+------------------+--------------------+-----------------+-----------------+-------+----------+-------+
| id|               line|           stop_name|borough|      lat|      long|           n_label|             s_label|          entries|            exits|arrests|complaints|summons|
+---+-------------------+--------------------+-------+---------+----------+------------------+--------------------+-----------------+-----------------+-------+----------+-------+
|F12|        Queens Blvd|          5 Av/53 St|      M|40.760167|-73.975224|            Queens| Downtown & Brooklyn| 1.71540057484E11| 1.83989511603E11|   1811|      3603|   1056|
|637|       Lexington Av|         Bleecker St|      M|40.725915|-73.994659|Uptown & The Bronx|            Downtown| 1.96565180833E11| 1.53466659135E11|   3444|      5079|   1046|
|603|             Pelham|       Middletown Rd|     Bx|40.843863|-73.836322|   Pelham Bay Park|           

### Conclusion

In [30]:
ss_df.toPandas().head()

,id,line,stop_name,borough,lat,long,n_label,s_label,entries,exits,arrests,complaints,summons
0,F12,Queens Blvd,5 Av/53 St,M,40.760167,-73.975224,Queens,Downtown & Brooklyn,1.715401e+11,1.839895e+11,1811,3603,1056
1,637,Lexington Av,Bleecker St,M,40.725915,-73.994659,Uptown & The Bronx,Downtown,1.965652e+11,1.534667e+11,3444,5079,1046
2,603,Pelham,Middletown Rd,Bx,40.843863,-73.836322,Pelham Bay Park,Manhattan,1.602757e+10,9.955782e+09,442,1094,842
3,725,Flushing,Times Sq-42 St,M,40.755477,-73.987691,Queens,34 St - Hudson Yards,4.943579e+11,1.840059e+11,11199,12406,14342
4,606,Pelham,Zerega Av,Bx,40.836488,-73.847036,Pelham Bay Park,Manhattan,9.471628e+09,5.313407e+09,649,1661,996
